In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [8]:
import tensorflow as tf

# Enable memory growth for all GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


# Load metadata
df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')

In [9]:
def get_image_path(image_id):
    p1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/"
    p2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/"
    return os.path.join(p1 if os.path.exists(p1 + image_id + ".jpg") else p2, image_id + ".jpg")

df["image_path"] = df["image_id"].apply(get_image_path)
labels = sorted(df["dx"].unique())
label_map = {l: i for i, l in enumerate(labels)}
df["label"] = df["dx"]

In [10]:
# Data split
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)

# Generators
IMG_SIZE = 224
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=15, horizontal_flip=True, zoom_range=0.2)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(train_df, x_col="image_path", y_col="label", target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=32)
val_data = val_gen.flow_from_dataframe(val_df, x_col="image_path", y_col="label", target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=32)

Found 8512 validated image filenames belonging to 7 classes.
Found 1503 validated image filenames belonging to 7 classes.


In [11]:
# Model builder
def build_model(base):
    base_model = base(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.3)(x)
    out = Dense(len(labels), activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=out)

In [12]:
# Train base models
models = [build_model(base) for base in [EfficientNetB0, MobileNetV2, ResNet50]]
for i, model in enumerate(models):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    print(f"\nTraining Model {i+1}")
    model.fit(train_data, validation_data=val_data, epochs=7)
    model.save(f"model/model_{i}.h5")

I0000 00:00:1747391660.549104      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747391660.549773      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step

Training Model 1


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7


I0000 00:00:1747391687.934114      98 service.cc:148] XLA service 0x7afcfc004530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747391687.935280      98 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747391687.935304      98 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747391689.910977      98 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/266 ━━━━━━━━━━━━━━━━━━━━ 13s 50ms/step - accuracy: 0.2578 - loss: 1.8399   

I0000 00:00:1747391699.426137      98 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


266/266 ━━━━━━━━━━━━━━━━━━━━ 228s 756ms/step - accuracy: 0.6519 - loss: 1.2127 - val_accuracy: 0.6693 - val_loss: 1.1397
Epoch 2/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 139s 523ms/step - accuracy: 0.6636 - loss: 1.1691 - val_accuracy: 0.6693 - val_loss: 1.1414
Epoch 3/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 139s 521ms/step - accuracy: 0.6749 - loss: 1.1301 - val_accuracy: 0.6693 - val_loss: 1.1377
Epoch 4/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 145s 543ms/step - accuracy: 0.6699 - loss: 1.1520 - val_accuracy: 0.6693 - val_loss: 1.1531
Epoch 5/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 142s 534ms/step - accuracy: 0.6704 - loss: 1.1548 - val_accuracy: 0.6693 - val_loss: 1.1382
Epoch 6/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 140s 528ms/step - accuracy: 0.6673 - loss: 1.1649 - val_accuracy: 0.6693 - val_loss: 1.1468
Epoch 7/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 143s 537ms/step - accuracy: 0.6746 - loss: 1.1425 - val_accuracy: 0.6693 - val_loss: 1.1345

Training Model 2
Epoch 1/7
266/266 ━━━━━━━━━━━━━━━━━━━━ 159s 557ms/step - accuracy: 0.6371 -

In [13]:
# Ensemble
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = [models[i](input_layer) for i in range(3)]
avg_output = Average()(outputs)
ensemble_model = Model(inputs=input_layer, outputs=avg_output)
ensemble_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ensemble_model.save("ensemble_skin_classifier.h5")
print("✅ Saved ensemble_skin_classifier.h5")

✅ Saved ensemble_skin_classifier.h5
